# Configuración

In [ ]:
# Configuración + Librerias

!pip install -Uq wandb

#librerias
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import v2
from sklearn.model_selection import train_test_split
import torch.utils.data
import numpy as np
import wandb
import torch
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

#login de wandb
wandb.login()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(7)
if torch.cuda.is_available():
    torch.cuda.manual_seed(7)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.9/243.9 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# Ejercicio 2

### Parámetros

In [ ]:
batch_size = 32
learning_rate = 0.02
momentum =.9
epochs = 10
project_name = "clasificación_img"

### Transformaciones

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

targets_ = trainset.targets
train_idx, val_idx = train_test_split(np.arange(len(targets_)), test_size=0.2, stratify=targets_)
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler,batch_size=batch_size, num_workers=2)
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler,batch_size=batch_size, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

100%|██████████| 170498071/170498071 [00:13<00:00, 13074900.30it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


KeyboardInterrupt: ignored

### Perceptron simple

In [ ]:
# Perceptron simple

class FC1(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 32*32)
        self.fc2 = nn.Linear(32*32, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# experiment_name = "3FC+ReLU"
# net = FC1()
# net.to(device)

In [ ]:
# Aumentamos la cantidad de nodos al perceptron simple

class FC2(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 32*32*6)
        self.fc2 = nn.Linear(32*32*6, 32*32)
        self.fc3 = nn.Linear(32*32, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# experiment_name = "3FC_2+ReLU"
# net = FC2()
# net.to(device)

FC2(
  (fc1): Linear(in_features=3072, out_features=6144, bias=True)
  (fc2): Linear(in_features=6144, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=10, bias=True)
)

### Perceptron Multicapa

In [ ]:
# A la red que mejor performance dio le aumentamos una capa

class FC3(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 32*32*6)
        self.fc21 = nn.Linear(32*32*6, 32*32*3)
        self.fc22 = nn.Linear(32*32*3, 32*32)
        self.fc3 = nn.Linear(32*32, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc21(x))
        x = F.relu(self.fc22(x))
        x = self.fc3(x)
        return x

# experiment_name = "4FC+ReLU"
# net = FC3()
# net.to(device)

FC3(
  (fc1): Linear(in_features=3072, out_features=6144, bias=True)
  (fc21): Linear(in_features=6144, out_features=3072, bias=True)
  (fc22): Linear(in_features=3072, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=10, bias=True)
)

In [ ]:
# volvemos a aumentar una capa mas

class FC4(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 32*32*6)
        self.fc21 = nn.Linear(32*32*6, 32*32*3)
        self.fc22 = nn.Linear(32*32*3, 32*32*3)
        self.fc23 = nn.Linear(32*32*3, 32*32)
        self.fc3 = nn.Linear(32*32, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc21(x))
        x = F.relu(self.fc22(x))
        x = F.relu(self.fc23(x))
        x = self.fc3(x)
        return x

# experiment_name = "5FC+ReLU"
# net = FC4()
# net.to(device)

FC4(
  (fc1): Linear(in_features=3072, out_features=6144, bias=True)
  (fc21): Linear(in_features=6144, out_features=3072, bias=True)
  (fc22): Linear(in_features=3072, out_features=3072, bias=True)
  (fc23): Linear(in_features=3072, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=10, bias=True)
)

In [ ]:
# volvemos a aumentar una capa mas

class FC5(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32*32*3, 32*32*6)
        self.fc21 = nn.Linear(32*32*6, 32*32*3)
        self.fc22 = nn.Linear(32*32*3, 32*32*3)
        self.fc23 = nn.Linear(32*32*3, 32*32*3)
        self.fc24 = nn.Linear(32*32*3, 32*32)
        self.fc3 = nn.Linear(32*32, 10)

    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc21(x))
        x = F.relu(self.fc22(x))
        x = F.relu(self.fc23(x))
        x = F.relu(self.fc24(x))
        x = self.fc3(x)
        return x

# experiment_name = "6FC+ReLU"
# net = FC5()
# net.to(device)

FC5(
  (fc1): Linear(in_features=3072, out_features=6144, bias=True)
  (fc21): Linear(in_features=6144, out_features=3072, bias=True)
  (fc22): Linear(in_features=3072, out_features=3072, bias=True)
  (fc23): Linear(in_features=3072, out_features=3072, bias=True)
  (fc24): Linear(in_features=3072, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=10, bias=True)
)

### Loss y optimización

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

### Entrenamiento

In [ ]:
wandb.init(
    project= project_name,
    entity = 'tp3-td6',
    name = experiment_name,

    # Hiperparametros
    config={
        "learning_rate": learning_rate,
        "momentum": momentum,
        "batch_size": batch_size,
        "epochs": epochs,
    }
)

start = time.time()

val_accuracies = []
train_losses = []
val_losses = []
best_val_accuracy = 0.0
best_epoch = 0

# loop over the dataset multiple times
for epoch in range(epochs):

    running_loss = 0.0
    train_correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):

        # Se obtinen los imputs (imagenes) y las labels
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()


    # End of test section
    # Val section
    train_accuracy = 100 * train_correct / total
    running_loss = running_loss/total

    val_correct = 0
    total =0
    val_loss =0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # End of test section

    val_accuracy = 100 * val_correct / total
    val_loss = val_loss / total

    val_accuracies.append(val_accuracy)

    train_losses.append(running_loss / total)
    val_losses.append(val_loss / total)

    if val_accuracy > best_val_accuracy:
        best_epoch = epoch
        best_val_accuracy = val_accuracy


    print(f"Guardando el modelo para la época {best_epoch}.")
    torch.save(net.state_dict(), f'{experiment_name}.pth')

    print('Epoch: {}/{}..'.format(epoch+1, epochs),
          'Training loss: {:.3f}..'.format(running_loss / total),
          'Val loss: {:.3f}..'.format(val_loss/ total),
          'Val accuracy: {:.3f}'.format(val_accuracy))

    wandb.log({ "train_accuracy": train_accuracy,
               "val_accuracy": val_accuracy,
                "train_loss": running_loss,
                "val_loss": val_loss,
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch })


end = time.time()
train_time = round(end - start)

print(f'Training time: {str(train_time)} seconds.')

wandb.log({'train_time': train_time})


print('Finished Training')

wandb.finish()

# Ejercicio 3

### Parámetros

In [ ]:
#Parametros
batch_size = 32
learning_rate = 0.02
momentum =.9
epochs = 10
project_name = "clasificación_img"

100%|██████████| 170498071/170498071 [00:03<00:00, 44058320.98it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### Transformaciones

In [ ]:
# Transformaciones
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

targets_ = trainset.targets
train_idx, val_idx = train_test_split(np.arange(len(targets_)), test_size=0.2, stratify=targets_)
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler,batch_size=batch_size, num_workers=2)
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler,batch_size=batch_size, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Redes Convolucionales

In [ ]:
# Red convolucional 1

class CNN1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 32, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

                            # 64 canales de 8x8 pues a la salida de cada capa
                            # convolucional se le aplica un maxpool de 2x2.
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)

        return x

# experiment_name = "2Conv+2FC+ReLU"
# net = CNN1()
# net.to(device)

CNN1(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# Red convolucional 2 - se le agrega una capa convolucional adicional

class CNN2(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 32, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 64,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)


                            # 64 canales de 8x8 pues a la salida de cada capa
                            # convolucional se le aplica un maxpool de 2x2.
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)

        return x

# experiment_name = "3Conv+2FC+ReLU"
# net = CNN2()
# net.to(device)

CNN2(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# Red convolucional 3 - se le agregan dos capas convolucionales adicionales

class CNN3(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 32, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 32,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 64,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv4 = nn.Conv2d(in_channels = 64,
                               out_channels = 64,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)


                            # 64 canales de 8x8 pues a la salida de cada capa
                            # convolucional se le aplica un maxpool de 2x2.
        self.fc1 = nn.Linear(64 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv4(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)

        return x

# experiment_name = "4Conv+2FC+ReLU"
# net = CNN3()
# net.to(device)

CNN3(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [ ]:
# Red convolucional 2 - A el modelo con la cantidad de capas convolucionales que mejor performance dió, se le aumenta la cantidad de canales de salida

class CNN4(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.ReLU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.ReLU()(self.fc1(x))
        x = self.fc2(x)

        return x

# experiment_name = "3Conv+2FC_2+ReLU"
# net = CNN4()
# net.to(device)

CNN5(
  (conv1): Conv2d(3, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(512, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=2048, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

### Loss y Optimizador

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

### Entrenamiento

In [ ]:
wandb.init(
    project= project_name,
    entity = 'tp3-td6',
    name = experiment_name,

    # Hiperparametros
    config={
        "learning_rate": learning_rate,
        "momentum": momentum,
        "batch_size": batch_size,
        "epochs": epochs,
    }
)

start = time.time()

val_accuracies = []
train_losses = []
val_losses = []
best_val_accuracy = 0.0
best_epoch = 0

# loop over the dataset multiple times
for epoch in range(epochs):

    running_loss = 0.0
    train_correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):

        # Se obtinen los imputs (imagenes) y las labels
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()


    # End of test section
    # Val section
    train_accuracy = 100 * train_correct / total
    running_loss = running_loss/total

    val_correct = 0
    total =0
    val_loss =0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # End of test section

    val_accuracy = 100 * val_correct / total
    val_loss = val_loss / total

    val_accuracies.append(val_accuracy)

    train_losses.append(running_loss / total)
    val_losses.append(val_loss / total)

    if val_accuracy > best_val_accuracy:
        best_epoch = epoch
        best_val_accuracy = val_accuracy


    print(f"Guardando el modelo para la época {best_epoch}.")
    torch.save(net.state_dict(), f'{experiment_name}.pth')

    print('Epoch: {}/{}..'.format(epoch+1, epochs),
          'Training loss: {:.3f}..'.format(running_loss / total),
          'Val loss: {:.3f}..'.format(val_loss/ total),
          'Val accuracy: {:.3f}'.format(val_accuracy))

    wandb.log({ "train_accuracy": train_accuracy,
               "val_accuracy": val_accuracy,
                "train_loss": running_loss,
                "val_loss": val_loss,
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch })


end = time.time()
train_time = round(end - start)

print(f'Training time: {str(train_time)} seconds.')

wandb.log({'train_time': train_time})


print('Finished Training')


wandb.finish()


# Ejercicio 4

### Parámetros

In [ ]:
#Parametros
batch_size = 32
learning_rate = 0.02
momentum =.9
epochs = 10
project_name = "clasificación_img"

### Transformaciones

In [ ]:
# Transformaciones
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

targets_ = trainset.targets
train_idx, val_idx = train_test_split(np.arange(len(targets_)), test_size=0.2, stratify=targets_)
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler,batch_size=batch_size, num_workers=2)
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler,batch_size=batch_size, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### LeakyReLU

In [ ]:
# Función de activación - Probamos usar LeakyReLU como función de activación en todas las capas

class CNN_LeakyReLU(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.LeakyReLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.LeakyReLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.LeakyReLU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.LeakyReLU()(self.fc1(x))
        x = self.fc2(x)

        return x

# experiment_name = "3Conv+2FC_2+LeakyReLU"
# net = CNN_LeakyReLU()
# net.to(device)

### Swish

In [ ]:
# Función de activación - Probamos usar SiLU como función de activación en todas las capas

class CNN_SiLU(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.SiLU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.SiLU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.SiLU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.SiLU()(self.fc1(x))
        x = self.fc2(x)

        return x

# experiment_name = "3Conv+2FC_2+SiLU"
# net = CNN_SiLU()
# net.to(device)

### GeLU

In [ ]:
# Función de activación - Probamos usar GELU como función de activación en todas las capas

class CNN_GELU(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.GELU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        x = self.fc2(x)

        return x

# experiment_name = "3Conv+2FC_2+GELU"
# net = CNN_GELU()
# net.to(device)

### Loss y Optimizador

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

### Entrenamiento

In [ ]:
wandb.init(
    project= project_name,
    entity = 'tp3-td6',
    name = experiment_name,

    # Hiperparametros
    config={
        "learning_rate": learning_rate,
        "momentum": momentum,
        "batch_size": batch_size,
        "epochs": epochs,
    }
)

start = time.time()

val_accuracies = []
train_losses = []
val_losses = []
best_val_accuracy = 0.0
best_epoch = 0

# loop over the dataset multiple times
for epoch in range(epochs):

    running_loss = 0.0
    train_correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):

        # Se obtinen los imputs (imagenes) y las labels
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()


    # End of test section
    # Val section
    train_accuracy = 100 * train_correct / total
    running_loss = running_loss/total

    val_correct = 0
    total =0
    val_loss =0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # End of test section

    val_accuracy = 100 * val_correct / total
    val_loss = val_loss / total

    val_accuracies.append(val_accuracy)

    train_losses.append(running_loss / total)
    val_losses.append(val_loss / total)

    if val_accuracy > best_val_accuracy:
        best_epoch = epoch
        best_val_accuracy = val_accuracy


    print(f"Guardando el modelo para la época {best_epoch}.")
    torch.save(net.state_dict(), f'{experiment_name}.pth')

    print('Epoch: {}/{}..'.format(epoch+1, epochs),
          'Training loss: {:.3f}..'.format(running_loss / total),
          'Val loss: {:.3f}..'.format(val_loss/ total),
          'Val accuracy: {:.3f}'.format(val_accuracy))

    wandb.log({ "train_accuracy": train_accuracy,
               "val_accuracy": val_accuracy,
                "train_loss": running_loss,
                "val_loss": val_loss,
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch })


end = time.time()
train_time = round(end - start)

print(f'Training time: {str(train_time)} seconds.')

wandb.log({'train_time': train_time})


print('Finished Training')


wandb.finish()


# Ejercicio 5

### Parámetros

In [ ]:
#Parametros
batch_size = 32
learning_rate = 0.0001
momentum =.9
epochs = 10
project_name = "clasificación_img"

### Transformaciones

In [ ]:
# Transformaciones
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

targets_ = trainset.targets
train_idx, val_idx = train_test_split(np.arange(len(targets_)), test_size=0.2, stratify=targets_)
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler,batch_size=batch_size, num_workers=2)
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler,batch_size=batch_size, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Red

In [ ]:
# Utilizamos el mejor modelo obtenido hasta el momento.

class EJ5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.GELU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        x = self.fc2(x)

        return x

### Loss y Optimizador

#### SGD

In [ ]:
# learning rate 0.02
# experiment_name = "3CNN+2FC+GELU+SGD"
# net = EJ5()
# net.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

#### Adagrad

In [ ]:
# learning rate 0.02
# experiment_name = "3CNN+2FC+GELU+Adagrad"
# net = EJ5()
# net.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adagrad(net.parameters(), lr=learning_rate)

#### Adam

In [ ]:
# learning rate 0.001
# experiment_name = "3CNN+2FC+GELU+Adam"
# net = EJ5()
# net.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
# learning rate 0.0001
# experiment_name = "3CNN+2FC+GELU+Adam_2"
# net = EJ5()
# net.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)

### Entrenamiento

In [ ]:
wandb.init(
    project= project_name,
    entity = 'tp3-td6',
    name = experiment_name,

    # Hiperparametros
    config={
        "learning_rate": learning_rate,
        "momentum": momentum,
        "batch_size": batch_size,
        "epochs": epochs,
    }
)

start = time.time()

val_accuracies = []
train_losses = []
val_losses = []
best_val_accuracy = 0.0
best_epoch = 0

# loop over the dataset multiple times
for epoch in range(epochs):

    running_loss = 0.0
    train_correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):

        # Se obtinen los imputs (imagenes) y las labels
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()


    # End of test section
    # Val section
    train_accuracy = 100 * train_correct / total
    running_loss = running_loss/total

    val_correct = 0
    total =0
    val_loss =0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # End of test section

    val_accuracy = 100 * val_correct / total
    val_loss = val_loss / total

    val_accuracies.append(val_accuracy)

    train_losses.append(running_loss / total)
    val_losses.append(val_loss / total)

    if val_accuracy > best_val_accuracy:
        best_epoch = epoch
        best_val_accuracy = val_accuracy


    print(f"Guardando el modelo para la época {best_epoch}.")
    torch.save(net.state_dict(), f'{experiment_name}.pth')

    print('Epoch: {}/{}..'.format(epoch+1, epochs),
          'Training loss: {:.3f}..'.format(running_loss / total),
          'Val loss: {:.3f}..'.format(val_loss/ total),
          'Val accuracy: {:.3f}'.format(val_accuracy))

    wandb.log({ "train_accuracy": train_accuracy,
               "val_accuracy": val_accuracy,
                "train_loss": running_loss,
                "val_loss": val_loss,
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch })


end = time.time()
train_time = round(end - start)

print(f'Training time: {str(train_time)} seconds.')

wandb.log({'train_time': train_time})


print('Finished Training')


wandb.finish()


[1,   200] loss: 0.189
[1,   400] loss: 0.343
[1,   600] loss: 0.488
[1,   800] loss: 0.624
[1,  1000] loss: 0.753
[1,  1200] loss: 0.875
Guardando el modelo para la época 0.
Epoch: 1/10.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 58.960
[2,   200] loss: 0.115
[2,   400] loss: 0.226
[2,   600] loss: 0.335
[2,   800] loss: 0.439
[2,  1000] loss: 0.541
[2,  1200] loss: 0.639
Guardando el modelo para la época 1.
Epoch: 2/10.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 65.560
[3,   200] loss: 0.092
[3,   400] loss: 0.183
[3,   600] loss: 0.269
[3,   800] loss: 0.357
[3,  1000] loss: 0.441
[3,  1200] loss: 0.527
Guardando el modelo para la época 2.
Epoch: 3/10.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 68.540
[4,   200] loss: 0.077
[4,   400] loss: 0.153
[4,   600] loss: 0.229
[4,   800] loss: 0.304
[4,  1000] loss: 0.378
[4,  1200] loss: 0.452
Guardando el modelo para la época 3.
Epoch: 4/10.. Training loss: 0.000.. Val loss: 0.000.. Val accuracy: 72.

best_epoch,▁▂▃▄▅▆▇███
best_val_accuracy,▁▄▅▇▇▇████
train_accuracy,▁▃▅▅▆▆▇▇██
train_loss,█▆▅▄▃▃▂▂▁▁
train_time,▁
val_accuracy,▁▄▅▇▇▇███▇
val_loss,█▅▃▂▁▁▁▁▂▂
best_epoch,7
best_val_accuracy,75.54
train_accuracy,90.8925
train_loss,0.00869


# Ejercicio 6

### Parámetros

In [ ]:
#Parametros
batch_size = 16 #32 #64
learning_rate = 0.0001
momentum =.9
epochs = 10 #30
project_name = "clasificación_img"

### Transformaciones

In [ ]:
# Transformaciones
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

targets_ = trainset.targets
train_idx, val_idx = train_test_split(np.arange(len(targets_)), test_size=0.2, stratify=targets_)
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler,batch_size=batch_size, num_workers=2)
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler,batch_size=batch_size, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

### Red

In [ ]:
# Utilizamos el mejor modelo obtenido hasta el momento.

class EJ5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.GELU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        x = self.fc2(x)

        return x

### Experimentos

In [ ]:
# learning rate 0.0001, batch size 16 y Epocas 10
# experiment_name = "3CNN+2FC+GELU+Adam_2+batch16+Epoch10"
# net = EJ5()
# net.to(device)

In [ ]:
# learning rate 0.0001, batch size 64 y Epocas 10
# experiment_name = "3CNN+2FC+GELU+Adam_2+batch64+Epoch10"
# net = EJ5()
# net.to(device)

In [ ]:
# learning rate 0.0001, batch size 64 y Epocas 30
# experiment_name = "3CNN+2FC+GELU+Adam_2+batch64+Epoch30"
# net = EJ5()
# net.to(device)

### Loss y Optimizador

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

### Entrenamiento

In [ ]:
wandb.init(
    project= project_name,
    entity = 'tp3-td6',
    name = experiment_name,

    # Hiperparametros
    config={
        "learning_rate": learning_rate,
        "momentum": momentum,
        "batch_size": batch_size,
        "epochs": epochs,
    }
)

start = time.time()

val_accuracies = []
train_losses = []
val_losses = []
best_val_accuracy = 0.0
best_epoch = 0

# loop over the dataset multiple times
for epoch in range(epochs):

    running_loss = 0.0
    train_correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):

        # Se obtinen los imputs (imagenes) y las labels
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()


    # End of test section
    # Val section
    train_accuracy = 100 * train_correct / total
    running_loss = running_loss/total

    val_correct = 0
    total =0
    val_loss =0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # End of test section

    val_accuracy = 100 * val_correct / total
    val_loss = val_loss / total

    val_accuracies.append(val_accuracy)

    train_losses.append(running_loss / total)
    val_losses.append(val_loss / total)

    if val_accuracy > best_val_accuracy:
        best_epoch = epoch
        best_val_accuracy = val_accuracy


    print(f"Guardando el modelo para la época {best_epoch}.")
    torch.save(net.state_dict(), f'{experiment_name}.pth')

    print('Epoch: {}/{}..'.format(epoch+1, epochs),
          'Training loss: {:.3f}..'.format(running_loss / total),
          'Val loss: {:.3f}..'.format(val_loss/ total),
          'Val accuracy: {:.3f}'.format(val_accuracy))

    wandb.log({ "train_accuracy": train_accuracy,
               "val_accuracy": val_accuracy,
                "train_loss": running_loss,
                "val_loss": val_loss,
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch })


end = time.time()
train_time = round(end - start)

print(f'Training time: {str(train_time)} seconds.')

wandb.log({'train_time': train_time})


print('Finished Training')


wandb.finish()


In [ ]:
wandb.finish()


# Ejercicio 7

### Parámetros

In [ ]:
batch_size = 64
learning_rate = 0.0001
momentum =.9
epochs = 30
project_name = "clasificación_img"

### Transformaciones

In [ ]:
# Descomentar la transformación que se desea aplicar.

train_transform = transforms.Compose([
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomRotation(degrees = (0,20)),
    # transforms.RandomCrop((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

val_test_transform = transforms.Compose([
    # Acá no aplicamos las otras transformaciones, porque ya no estamos en entrenamiento.
    transforms.ToTensor(),
    transforms.Normalize(0, 1)
])

In [ ]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)

targets_ = trainset.targets
train_idx, val_idx = train_test_split(np.arange(len(targets_)), test_size=0.2, stratify=targets_)
train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)

trainloader = torch.utils.data.DataLoader(trainset, sampler=train_sampler,batch_size=batch_size, num_workers=2)
valloader = torch.utils.data.DataLoader(trainset, sampler=val_sampler,batch_size=batch_size, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=val_test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


### Red convolucional original

In [ ]:
# 1. Probamos regularizando con nuestra red convolucional del punto anterior (con data augmentation y dropout)

class EJ5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.GELU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        x = self.fc2(x)

        return x

### Red con Dropout

In [ ]:
class EJ5(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels = 3,   # 1 canal de entrada.
                               out_channels = 256, # 32 canales de salida.
                               kernel_size = 3,   # Kernel 3x3.
                               stride = 1,        # Pasos de 1.
                               padding = 1)       # Padding de 1 para mantener el tamaño.

        self.conv2 = nn.Conv2d(in_channels = 256,
                               out_channels = 512,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.conv3 = nn.Conv2d(in_channels = 512,
                               out_channels = 128,
                               kernel_size = 3,
                               stride = 1,
                               padding = 1)

        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 10)
        # self.dropout = nn.Dropout(0.3)

    def forward(self, x):
        x = nn.GELU()(self.conv1(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv2(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = nn.GELU()(self.conv3(x))
        x = nn.MaxPool2d(kernel_size = 2, stride = 2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        # x = self.dropout(x)
        x = self.fc2(x)

        return x

### Red con Batch Norm

In [ ]:
# 3. Agregamos la barch normalization ya que VGG la usa y podria mejorar nuesta performance (hay que ver si dejamos el dropout o no)

class EJ5_batch(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(256)  # Agrega una capa de BatchNorm2d después de la convolución
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)  # Agrega una capa de BatchNorm2d después de la convolución
        self.conv3 = nn.Conv2d(in_channels=512, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)  # Agrega una capa de BatchNorm2d después de la convolución
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.GELU()(self.bn1(self.conv1(x)))  # Aplica BatchNorm después de la convolución
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.GELU()(self.bn2(self.conv2(x)))  # Aplica BatchNorm después de la convolución
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.GELU()(self.bn3(self.conv3(x)))  # Aplica BatchNorm después de la convolución
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        x = self.fc2(x)
        return x


### Red con batch y dropout

In [ ]:
class EJ5_batch_dropout(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(256)
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(512)
        self.conv3 = nn.Conv2d(in_channels=512, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 4 * 4, 128)
        self.dropout = nn.Dropout(0.2)  # Capa de dropout con probabilidad 0.2
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = nn.GELU()(self.bn1(self.conv1(x)))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.GELU()(self.bn2(self.conv2(x)))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = nn.GELU()(self.bn3(self.conv3(x)))
        x = nn.MaxPool2d(kernel_size=2, stride=2)(x)
        x = torch.flatten(x, 1)
        x = nn.GELU()(self.fc1(x))
        x = self.dropout(x)  # Aplicar dropout a la salida de la primera capa lineal
        x = self.fc2(x)
        return x


### Experimentos

In [ ]:
# agregamos horizontal flip con probabilidad default.
# experiment_name = "3CNN+2FC+GELU+Adam_2+HFlip"
# net = EJ5()
# net.to(device)

In [ ]:
# Probamos dropout sin data augmentation
# experiment_name = "3CNN+2FC+GELU+Adam_2+Dropout"
# net = EJ5()
# net.to(device)

In [ ]:
# Probamos Random Rotation
# experiment_name = "3CNN+2FC+GELU+Adam_2+Rotation"
# net = EJ5()
# net.to(device)

In [ ]:
# Probamos Random Crop
# experiment_name = "3CNN+2FC+GELU+Adam_2+Crop"
# net = EJ5()
# net.to(device)

In [ ]:
# agregamos rotacion de 0 a 20.
# experiment_name = "3CNN+2FC+GELU+Adam_2+HFlip+Rotation"
# net = EJ5()
# net.to(device)

### Experimentación extra

In [ ]:
# Probamos con batch norm
# experiment_name = "3CNN+2FC+GELU+Adam_2+BatchNorm"
# net = EJ5_batch()
# net.to(device)

In [ ]:
# Probamos con batch norm y dropout de 0.2
# experiment_name = "3CNN+2FC+GELU+Adam_2+BatchNorm+Dropout"
# net = EJ5_batch_dropout()
# net.to(device)

### Loss y Optimizador

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=learning_rate)

### Entrenamiento

In [ ]:
wandb.init(
    project= project_name,
    entity = 'tp3-td6',
    name = experiment_name,

    # Hiperparametros
    config={
        "learning_rate": learning_rate,
        "momentum": momentum,
        "batch_size": batch_size,
        "epochs": epochs,
    }
)

start = time.time()

val_accuracies = []
train_losses = []
val_losses = []
best_val_accuracy = 0.0
best_epoch = 0

# loop over the dataset multiple times
for epoch in range(epochs):

    running_loss = 0.0
    train_correct = 0
    total = 0

    for i, data in enumerate(trainloader, 0):

        # Se obtinen los imputs (imagenes) y las labels
        inputs = data[0].to(device)
        labels = data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 200 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')


        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        train_correct += (predicted == labels).sum().item()


    # End of test section
    # Val section
    train_accuracy = 100 * train_correct / total
    running_loss = running_loss/total

    val_correct = 0
    total =0
    val_loss =0

    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in valloader:
            images, labels = data[0].to(device), data[1].to(device)
            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
            val_loss += criterion(outputs, labels).item()

    # End of test section

    val_accuracy = 100 * val_correct / total
    val_loss = val_loss / total

    val_accuracies.append(val_accuracy)

    train_losses.append(running_loss / total)
    val_losses.append(val_loss / total)

    if val_accuracy > best_val_accuracy:
        best_epoch = epoch
        best_val_accuracy = val_accuracy


    print(f"Guardando el modelo para la época {best_epoch}.")
    torch.save(net.state_dict(), f'{experiment_name}.pth')

    print('Epoch: {}/{}..'.format(epoch+1, epochs),
          'Training loss: {:.3f}..'.format(running_loss / total),
          'Val loss: {:.3f}..'.format(val_loss/ total),
          'Val accuracy: {:.3f}'.format(val_accuracy))

    wandb.log({ "train_accuracy": train_accuracy,
               "val_accuracy": val_accuracy,
                "train_loss": running_loss,
                "val_loss": val_loss,
                "best_val_accuracy": best_val_accuracy,
                "best_epoch": best_epoch })


end = time.time()
train_time = round(end - start)

print(f'Training time: {str(train_time)} seconds.')

wandb.log({'train_time': train_time})


print('Finished Training')


wandb.finish()


# Ejercicio 8

##### El ejercicio 8 lo resolvimos en otro collab dado que ninguna de nuestras computadoras estaba pudiendo conectarse a la GPU del collab. La unica forma que se nos ocurrio para correr el testeo fue crear otro collab para usar esa GPU.